# 12wk-1: ???

최규빈  
2023-11-20

<a href="https://colab.research.google.com/github/guebin/DV2023/blob/main/posts/12wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-wFUcEr7JtSb7MWUYl6QrW_&si=CmWA765p0mnwWTGA>